In [14]:
import tensorflow as tf
import numpy as np
from keras import models, layers, datasets
import matplotlib.pyplot as plt
import os
from PIL import Image
import random

Load the pretrained MobileNetV2 model and the ImageNet class names

In [15]:
pretrained_model = models.load_model("models/MobileNet-FineTuned-CIFAR10.h5")
pretrained_model.trainable = False

In [16]:
# Preprocess the input image for your fine-tuned model
def preprocess(image):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (32, 32))  
    image = image / 255.0  
    image = image[None, ...]  
    return image

# Extract label from probability vector
def get_cifar10_label(probs):
    return np.argmax(probs)

In [17]:
(X_train,y_train),(X_test,y_test) = datasets.cifar10.load_data()

## Creating Adversarial Image

In [18]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

def create_adversarial_pattern(input_image, input_label):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = pretrained_model(input_image)
    loss = loss_object(input_label, prediction)

  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)
  # Get the sign of the gradients to create the perturbation
  signed_grad = tf.sign(gradient)
  return signed_grad

In [19]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [20]:
epsilons = [0.01]

num_samples =  50 #len(X_test[:200])
mobilenet_pred_labels = []
adversarial_pred_labels = []

correct_clean = 0
incorrect_clean = 0
correct_adv = 0
incorrect_adv = 0

for idx in range(num_samples):
    # Process the original image
    image = preprocess(X_test[idx])
    image_probs = pretrained_model.predict(image)
    predicted_label = np.argmax(image_probs, axis=-1)
    mobilenet_pred_labels.append(predicted_label)
  
    true_label = y_test[idx]

    # Check accuracy on clean images
    if predicted_label == true_label:
        correct_clean += 1
    else:
        incorrect_clean += 1

    label = tf.one_hot(true_label, image_probs.shape[-1])
    label = tf.reshape(label, (1, image_probs.shape[-1]))

    # Generate perturbations
    perturbations = create_adversarial_pattern(image, label)
    
    # Save perturbations as an image
    perturbation_img = (perturbations.numpy()[0] * 0.5 + 0.5) * 255  # Scale to 0-255 range
    perturbation_img = perturbation_img.astype(np.uint8)
    perturbation_img_pil = Image.fromarray(perturbation_img)
    perturbation_img_pil.save(f"images/SingleEps/test_perturbations/{idx}.png")

    # Generate and save adversarial image with a random epsilon
    random_eps = random.choice(epsilons)
    adv_x = image + random_eps * perturbations
    adv_x = tf.clip_by_value(adv_x, 0, 1)  # Clip to [0, 1] range
    
    # Convert adversarial image to numpy array and scale to 0-255 range
    adv_x_np = (adv_x.numpy()[0] * 255).astype(np.uint8)
    adv_x_pil = Image.fromarray(adv_x_np)
    adv_x_pil.save(f"images/SingleEps/test_adversarial/{idx}.png")

    # Predict on adversarial image
    adv_image_probs = pretrained_model.predict(adv_x)
    adv_predicted_label = np.argmax(adv_image_probs, axis=-1)
    adversarial_pred_labels.append(adv_predicted_label)
    
    # Check accuracy on adversarial images
    if adv_predicted_label == true_label:
        correct_adv += 1
    else:
        incorrect_adv += 1

# Print results
print(f"Clean Image Predictions: Correct = {correct_clean}, Incorrect = {incorrect_clean}")
print(f"Adversarial Image Predictions: Correct = {correct_adv}, Incorrect = {incorrect_adv}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

# Defence Pipeline

## Loading Denoising Autoencoder

In [21]:
from tensorflow.keras.losses import MeanSquaredError

# Load the denoising autoencoder model with custom objects
dae = models.load_model("models/DenoisingAutoEncoder/[0.07]-DAE-CIFAR10.h5", custom_objects={'mse': MeanSquaredError()})

In [22]:
# Counters for denoised image predictions
correct_denoised = 0
incorrect_denoised = 0

for idx in range(num_samples):
    # Load adversarial image
    adv_image_path = f"images/SingleEps/test_adversarial/{idx}.png"
    adv_image = Image.open(adv_image_path)
    adv_image = np.array(adv_image) / 255.0  # Normalize to [0, 1] range
    adv_image = np.expand_dims(adv_image, axis=0)  # Add batch dimension

    # Denoise the adversarial image using the DAE
    denoised_image = dae.predict(adv_image)
    
    # Save denoised image
    denoised_image_np = (denoised_image[0] * 255).astype(np.uint8)
    denoised_image_pil = Image.fromarray(denoised_image_np)
    denoised_image_pil.save(f"images/SingleEps/test_denoised/{idx}.png")

    # Preprocess the denoised image for MobileNet prediction
    preprocessed_denoised_image = preprocess(denoised_image[0])
    
    # Predict on denoised image using MobileNet
    denoised_image_probs = pretrained_model.predict(preprocessed_denoised_image)
    denoised_predicted_label = get_cifar10_label(denoised_image_probs)
    
    true_label = y_test[idx]
    
    # Check accuracy on denoised images
    if denoised_predicted_label == true_label:
        correct_denoised += 1
    else:
        incorrect_denoised += 1

# Print results for denoised images
print(f"Denoised Image Predictions: Correct = {correct_denoised}, Incorrect = {incorrect_denoised}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━

## Load BlockSwitch Model

In [23]:
# Define the custom layer
class RandomSwitch(layers.Layer):
    def __init__(self, num_choices, **kwargs):
        super(RandomSwitch, self).__init__(**kwargs)
        self.num_choices = num_choices

    def call(self, inputs, training=None):
        inputs = tf.stack(inputs, axis=0)
        if training:
            choice = tf.random.uniform(shape=[], minval=0, maxval=self.num_choices, dtype=tf.int32)
            return inputs[choice]
        else:
            return tf.reduce_mean(inputs, axis=0)

# Load the BlockSwitching model with the custom layer
custom_objects = {'RandomSwitch': RandomSwitch}
block_switching_model = models.load_model("models/BlockSwitching/cifar10_block_switching_model.h5", custom_objects=custom_objects)


In [24]:
# Path to the directory with denoised images
denoised_images_path = "images/SingleEps/test_denoised/"

# Initialize variables to store cumulative accuracy
total_samples = len(os.listdir(denoised_images_path))
correct_predictions = 0
incorrect_predictions = 0

# Evaluation loop
for idx in range(total_samples):
    # Load the denoised image
    denoised_image = Image.open(os.path.join(denoised_images_path, f"{idx}.png"))
    denoised_image = np.array(denoised_image)
    
    # Preprocess the denoised image
    denoised_image = preprocess(denoised_image)
    
    # Predict the label using BlockSwitching model
    test_logits = block_switching_model(denoised_image, training=False)
    predicted_label = np.argmax(test_logits, axis=-1)
    
    # Get the true label
    true_label = y_test[idx]
    
    # Compare the predicted label with the true label
    if predicted_label == true_label:
        correct_predictions += 1
    else:
        incorrect_predictions += 1

# Print the accuracy
accuracy = correct_predictions / total_samples
print(f"Total Samples: {total_samples}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Incorrect Predictions: {incorrect_predictions}")
print(f"Accuracy: {accuracy:.4f}")

Total Samples: 50
Correct Predictions: 18
Incorrect Predictions: 32
Accuracy: 0.3600


## Direclty Using Adversarial Images for BlockSwitching Model without Denoising them

In [25]:
# Define the custom layer
class RandomSwitch(layers.Layer):
    def __init__(self, num_choices, **kwargs):
        super(RandomSwitch, self).__init__(**kwargs)
        self.num_choices = num_choices

    def call(self, inputs, training=None):
        inputs = tf.stack(inputs, axis=0)
        if training:
            choice = tf.random.uniform(shape=[], minval=0, maxval=self.num_choices, dtype=tf.int32)
            return inputs[choice]
        else:
            return tf.reduce_mean(inputs, axis=0)

# Load the BlockSwitching model with the custom layer
custom_objects = {'RandomSwitch': RandomSwitch}
block_switching_model = models.load_model("models/BlockSwitching/cifar10_block_switching_model.h5", custom_objects=custom_objects)


In [26]:
# Path to the directory with adversarial images
adversarial_images_path = "images/SingleEps/test_adversarial/"

# Function to preprocess images for BlockSwitching model
def preprocess_for_block_switching(image):
    image = tf.image.resize(image, (32, 32))  # Resize to match input size of BlockSwitching model
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    return image

# Initialize variables to count correct and incorrect predictions
total_samples = len(os.listdir(adversarial_images_path))
correct_predictions = 0
incorrect_predictions = 0

# Loop through adversarial images and evaluate predictions
for filename in os.listdir(adversarial_images_path):
    # Load and preprocess the image
    image_path = os.path.join(adversarial_images_path, filename)
    img = Image.open(image_path)
    image = np.array(img)
    image = preprocess_for_block_switching(image)
    image = tf.expand_dims(image, axis=0)  # Add batch dimension
    
    # Predict using BlockSwitching model
    logits = block_switching_model.predict(image)
    predicted_label = np.argmax(logits, axis=-1)
    
    # Determine if prediction is correct
    if predicted_label == true_label:  # Replace true_label with actual true label variable
        correct_predictions += 1
    else:
        incorrect_predictions += 1

# Print results
print(f"Total correct predictions: {correct_predictions}")
print(f"Total incorrect predictions: {incorrect_predictions}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 650ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━